### Многопоточность. Atomics.

<br />

Документация:
* https://en.cppreference.com/w/cpp/atomic
* https://en.cppreference.com/w/cpp/atomic/memory_order

Серия статей от Jeff Pershing на понимание atomics, memory model && lock free:
* https://preshing.com/20120515/memory-reordering-caught-in-the-act/
* https://preshing.com/20120612/an-introduction-to-lock-free-programming
* https://preshing.com/20120625/memory-ordering-at-compile-time/
* https://preshing.com/20120710/memory-barriers-are-like-source-control-operations/
* https://preshing.com/20120913/acquire-and-release-semantics/
* https://preshing.com/20120930/weak-vs-strong-memory-models/
* https://preshing.com/20121019/this-is-why-they-call-it-a-weakly-ordered-cpu/
* https://preshing.com/20130618/atomic-vs-non-atomic-operations/
* https://preshing.com/20130702/the-happens-before-relation/
* https://preshing.com/20130922/acquire-and-release-fences/
* https://preshing.com/20130823/the-synchronizes-with-relation/
* https://preshing.com/20130930/double-checked-locking-is-fixed-in-cpp11/
* https://preshing.com/20131125/acquire-and-release-fences-dont-work-the-way-youd-expect/
* https://preshing.com/20140709/the-purpose-of-memory_order_consume-in-cpp11/
* В конце своих статей Джефф даёт ссылки на полезные материалы, их тоже рекомендуется почитать. Не стоит рассчитывать, что управитесь со всем багажом знаний за полчасика.
* И в комментарии к статьям приходят специалисты (Herb Sutter, Tarvis Downs), и объясняют, в чём Джефф был не прав, поэтому комментарии желательно тоже смотреть.

Другие статьи:
* [Memory Barriers: a Hardware View for Software Hackers](http://www.rdrop.com/users/paulmck/scalability/paper/whymb.2010.07.23a.pdf)
* [LINUX KERNEL MEMORY BARRIERS](https://www.kernel.org/doc/Documentation/memory-barriers.txt)
* [Максим Хижинский. Lock-free Data Structures. Basics: Atomicity and Atomic Primitives](https://kukuruku.co/post/lock-free-data-structures-basics-atomicity-and-atomic-primitives/)
* [Максим Хижинский. C++ Siberia 2019: Максим Хижинский, Hazard Pointer внутри и снаружи](https://www.youtube.com/watch?v=aczfcRhXrzI)
* [Bjarne Stroustrup про ABA-проблему и как её решать](http://www.stroustrup.com/isorc2010.pdf)

Доклады про многопоточность:
* [CppCon 2017: Fedor Pikus “C++ atomics, from basic to advanced. What do they really do?”](https://www.youtube.com/watch?v=ZQFzMfHIxng)
* [CppCon 2018: Bryce Adelstein Lelbach “The C++ Execution Model”](https://www.youtube.com/watch?v=FJIn1YhPJJc)
* [CppCon 2019: Bryce Adelstein Lelbach “The C++20 Synchronization Library”](https://www.youtube.com/watch?v=Zcqwb3CWqs4)

##### atomic basics (since C++11)

Вспомним задачу параллельного суммирования массива из второй лекции на многопоточность.

Её реализация с ошибкой:

```c++
int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    int rv = 0;

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([&v, &rv, i, len](){
            for (unsigned ix = len * i, final_ix = len * (i + 1); ix < final_ix; ++ix)
                 rv += v[ix];
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```

**Вопрос**: напомните, в чём ошибка?

<br />

Во второй лекции проблему решали через `mutex`.

**Вопрос**: как лучше всего решить задачу, чтобы синхронизаций было поменьше?

Сейчас будем писать тоже неидеальное решение - через `std::atomic`

Атомарность операции означает, что никакой из потоков не может отследить промежуточное состояние операции: либо состояние до изменений, либо состояние после изменений.

**Пример:**

`std::atomic<int>` имеет операцию `fetch_add`:

`var.fetch_add(int x)` имеет три условных стадии выполнения:

1. считать текущее значение из памяти в регистр
2. увеличить регистр на значение `x`
3. записать значение регистра в память

Никакой поток не может вклиниться в работу с `var`, пока выполняются шаги 1-2-3. Он либо работает с `var` до шага 1, либо после шага 3.


**Вопрос:** в примере ниже:
1. какое значение `counter` возможно после того как потоки выполняют свою работу?
2. если бы гарантии атомарности не было, какое значение мы могли бы получить?

```c++
std::atomic<int> counter{0};

void thread_1_worker() { counter.fetch_add(1); }
void thread_2_worker() { counter.fetch_add(1); }

std::thread t1(thread_1_worker), t2(thread_2_worker);
t1.join(); t2.join();
```


Из документации по типу `atomic<T>`:

> Each instantiation and full specialization of the `std::atomic` template defines an atomic type. If one thread writes to an atomic object while another thread reads from it, the behavior is well-defined (see memory model for details on data races)

Какие типы `T` можно подставлять?

Обратимся опять к документации:

> The primary `std::atomic` template may be instantiated with any `TriviallyCopyable` type `T` satisfying both `CopyConstructible` and `CopyAssignable`. The program is ill-formed if any of following values is false:
* `std::is_trivially_copyable<T>::value`
* `std::is_copy_constructible<T>::value`
* `std::is_move_constructible<T>::value`
* `std::is_copy_assignable<T>::value`
* `std::is_move_assignable<T>::value`

Т.е. для любого `TriviallyCopyable` типа `T` объект типа `std::atomic<T>` потокобезопасен на чтение и запись.

```c++
struct Point
{
    float x;
    float y;
    float z;
};

std::atomic<Point> p;

void thread_1_worker() {
    p = Point{1.f, 2.f, 3.f};  // ok
}

void thread_2_worker() {
    Point x = p;  // ok
}
```

А вот эти примеры - ill-formed:

```c++
std::atomic<std::string> s;  // ill-formed
std::atomic<std::vector<int>> v;  // ill-formed
```

<br />

##### atomic vs mutex: performance

Для каких-то сложных типов (например, `Point`) `std::atomic` реализуется через `std::mutex` или аналогичным образом.

Если бы так было для всех типов, то особого смысла в `std::atomic` бы не было.

Смысл в том, что для простых типов (`int`, `bool`, `int64_t` ...) многие CPU поддерживают более дешёвые способы синхронизации. Набор типов и степень их дешевизны зависят от архитектуры CPU.

Напишем реализацию паралелльной суммы через `std::atomic<int>`:

```c++
int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    std::atomic<int> rv{0};

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([&v, &rv, i, len](){
            for (unsigned ix = len * i, final_ix = len * (i + 1); ix < final_ix; ++ix)
                 rv.fetch_add(v[ix]);
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```

И через `std::mutex`:

```c++
int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    int rv = 0;
    std::mutex m;

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([&v, &rv, &m, i, len](){
            for (unsigned ix = len * i, final_ix = len * (i + 1); ix < final_ix; ++ix)
            {
                std::lock_guard guard(m);
                rv += v[ix];                
            }
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```

И сравним производительность на 6 потоках (тестовая машинка: 6 физических ядер Intel Core i5-8400):

```sh
g++ parallel_sum_atomic.cpp -lpthread -O3 -std=c++17 -o sum_atomic.exe && ./sum_atomic.exe
g++ parallel_sum_mutex.cpp  -lpthread -O3 -std=c++17 -o sum_mutex.exe  && ./sum_mutex.exe
```

вывод:

```sh
parallel sum atomic:
  size          = 60000000
  threads_count = 6
  result        = 60000000
  time, sec     = 1.60537
 
parallel sum mutex:
  size          = 60000000
  threads_count = 6
  result        = 60000000
  time, sec     = 6.33526
```

В зависимости от примитива синхронизации получилось добиться 4-х кратного ускорения алгоритма, что существенно.
Такой эффект достигается за счёт того, что в алгоритме слишком агрессивно используется синхронизация данных.

<br />

Чтобы оценить что происходит в бинарном коде закинем файл на godbolt.org (gcc 9.2 -O3 -std=c++17):

```c++
#include <atomic>
#include <mutex>

int mutexed_counter = 0;
std::mutex m;

std::atomic<int> atomic_counter{0};

void add_mutexed(int value)
{
    std::lock_guard guard{m};
    mutexed_counter += value;
}

void add_atomic(int value)
{
    atomic_counter += value;
}
```

**Вопрос**: Перед тем как посмотреть на ответ, подскажите, что происходит внутри `lock_guard` и `mutex` в этом коде?

```asm
add_mutexed(int):
        push    rbp
        mov     ebp, OFFSET FLAT:_ZL28__gthrw___pthread_key_createPjPFvPvE
        push    rbx
        mov     ebx, edi
        sub     rsp, 8
        test    rbp, rbp
        je      .L2
        mov     edi, OFFSET FLAT:m
        call    __gthrw_pthread_mutex_lock(pthread_mutex_t*)
        test    eax, eax
        jne     .L12
.L2:
        add     DWORD PTR mutexed_counter[rip], ebx
        test    rbp, rbp
        je      .L1
        add     rsp, 8
        mov     edi, OFFSET FLAT:m
        pop     rbx
        pop     rbp
        jmp     __gthrw_pthread_mutex_unlock(pthread_mutex_t*)
.L1:
        add     rsp, 8
        pop     rbx
        pop     rbp
        ret
.L12:
        mov     edi, eax
        call    std::__throw_system_error(int)
        
add_atomic(int):
        lock add        DWORD PTR atomic_counter[rip], edi
        ret
```

Внутри `add_mutexed` можно наблюдать уход в ядро ОС через `pthread`.

А весь `add_atomic` - одна инструкция `lock add` - особая атомарная инструкция сложения.

<br />

##### atomics: hardware

Сначала рассмотрим hardware-нюансы обзорно.

Вспомним многоуровневую организацию кешей памяти:

<img src="cpu_caches_ram.png" width=50% height=50% />

При работе многопоточного приложения одна и та же ячейка памяти может оказаться одновременно в разных кешах L1 разных CPU.

Если при этом происходит только чтение из памяти, то проблем не возникает. Возникают проблемы, когда нужна запись.

Записанное значение должно "просочиться" по всем уровням иерархии кешей вплоть до RAM... и обновиться в кешах соседних CPU.

Процесс "просачивания" не быстрый, другие CPU могут читать устаревшее значение или попытаться записать своё.

Особые atomic-инструкции решают эту проблему, они гарантируют, что записанное одним CPU значение "просочится" по всей иерархии кешей и будет корректно прочитано другими CPU.

Поэтому атомарные инструкции (как правило - зависит от железа) медленнее аналогичных неатомарных инструкций:
  * `+=` для `std::atomic<int>` будет медленнее чем для `int`
  * также работа с `std::atomic<int>` отключает некоторые оптимизации компилятора (подробности в разделе про instruction reordering && memory model)

Это упрощённое описание. Пока что его будет достаточно для дальнейшей работы. Нюансы о гарантиях и оптимизации atomic - раздел про memory model, о нём позже.

<br />

##### instructions reordering

* https://preshing.com/20120625/memory-ordering-at-compile-time/
* https://preshing.com/20120710/memory-barriers-are-like-source-control-operations/

Переупорядочивание инструкций - изменение порядка выполняемых команд "без видимых эффектов" - одна из оптимизаций программ, будучи полностью прозрачной для однопоточного кода, начинает играть роль для некорректного многопоточного кода (для корректного всё хорошо).

Важно, что _ассемблерные_ инструкции не могут быть переупорядочены, если между ними есть зависимость по данным.

Переупорядочивание инструкций можно разделить на 2 типа:
* в compile time
* в runtime

<br />

**compile-time**

Во-первых, компилятор может решить, что лучше команды выполнять не в той последовательности, в которой их описал программист.

Рассмотрим такой код:

```c++

double some_value = 0.;
bool value_is_set = false;


void run_setup()
{
    some_value = 3.14;    // между some_value и value_is_set нет зависимости
    value_is_set = true;  // по данным, компилятор вправе поменять присванивания местами
}

void process()
{
    if (value_is_set)
        assert(some_value == 3.14);
}
```

В однопоточном приложении здесь всё будет хорошо, а в многопоточном и так плохо т.к. нет нужных синхронизаций, но переупорядочивание инструкций может увеличить вероятность ошибки.


Другой пример с переупорядочиванием инструкций, когда зависимость по данным есть в плюсовом коде, но её нет на увроне инструкций ассемблера:

```c++
int A, B;

void foo()
{
    A = B + 1;
    B = 0;
}
```

Здесь компилятор так же вправе сгенерировать запись в ячейку памяти B раньше чем в A:

либо так:

```asm
mov     eax, DWORD PTR _B
add     eax, 1
mov     DWORD PTR _A, eax
mov     DWORD PTR _B, 0
```

либо так:

```asm
mov     eax, DWORD PTR B
mov     DWORD PTR B, 0
add     eax, 1
mov     DWORD PTR A, eax
```

Опять же, для однопоточного кода такие трюки должны быть прозрачны. Они начинают играть свою роль только для многопоточного кода.

**Замечание**: есть способ запретить копилятору переупорядочивать некоторые операции, добавив барьер, [подробнее - читайте по ссылке выше](https://preshing.com/20120625/memory-ordering-at-compile-time/)

<br />

**runtime**

Во-вторых, даже если компилятор сохранил последовательность команд программиста, CPU может иметь своё мнение, и решить, что быстрее будет их выполнить в ином порядке.

Рассмотрим пример такого устройства:

![](cpu_mem_organization.jpg)

В этой схеме у каждого ядра "своя личная область работы" - кеш L1, а "общая область" - кеш L2 + RAM.

Рассмотрим случай, когда в какой-то программе поток 1 выполняет код на ядре 1, а поток 2 выполняет код на ядре 2:

```c++
int x = 0, y = 0;
int a = 0, b = 0;

std::thread t1([&]() {
    x = 1; y = 2;
    
    if (b == 5)
        assert(a == 4);  // might fail
});

std::thread t2([&]() {
    a = 4; b = 5;

    if (y == 2)
        assert(x == 1);  // might fail
});

t1.join();
t2.join();
```

В таком коде нет гарантий на порядок записи / чтения между "личной областью работы" и "общей областью работы".

* Порядок, кто из `x`, `y` раньше попадёт в L2 + RAM не определён, так же как и время, через которое они туда попадут.
  * вполне может оказаться так, что y улетит в RAM первым, а x ещё долго провисят недоставленными по адресу в L1
* Аналогично, не определён порядок попадания `a`, `b` в "общую область работы"
* Не определён порядок попадания `x`, `y` в "личную область работы" потока 2, т.е. при чтении тоже нет гарантий, что поток 2, увидев `y == 2`, получит `x == 1`
* Если запустить поток 3 на cpu 3, он может увидеть совсем другую последовательность значений `x`, `y` чем поток 2

При таких условиях многопоточных код писать невозможно. Чтобы справляться с этими проблемами есть особые барьеры памяти (их много и разных видов), о них поговорим чуть позже.

Если коротко, без использования барьеров памяти:
* значения на запись уходят в "общую область" сколь угодно долго и в произвольном порядке
* значения на чтение приходят в "личную область" сколько угодно долго и в произвольном порядке

<br />

##### double checked locking

https://preshing.com/20130930/double-checked-locking-is-fixed-in-cpp11/

http://www.cs.umd.edu/~pugh/java/memoryModel/DoubleCheckedLocking.html

https://www.aristeia.com/Papers/DDJ_Jul_Aug_2004_revised.pdf

DCLP - double checked locking pattern - популярный шаблон организации потокобезопасного доступа к данным/объектам с ленивой инициализацией. Пример: синглтоны с ленивой инициализацией.

**Вопрос-напоминалка**: что такое синглтон?

Однопоточный вариант организации ленивого синглтона:
    
```c++
class Singleton
{
    static Singleton* object;

public:
    static Singleton* instance()
    {
        if (!object)
            object = new Singleton;
        return object;
    }
};
```

**Вопрос**: Почему эта реализация не работает в многопоточной среде? Приведите пример последовательности выполнения потоков, при котором возникает ошибка.

<br />

Многопоточный вариант реализации ленивого синглтона:
    
```c++
class Singleton
{
    static Singleton* object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        std::lock_guard guard(mtx);
        
        if (!object)
            object = new Singleton;
        return object;
    }
};
```

**Вопрос**: корректна ли эта реализация? Какие у неё проблемы?

<details>
<summary>ответ</summary>
<p>

Корректна, но очень дорого: когда синглтон инициализирован каждый из потоков спотыкается об `mutex` при доступе к синглтону. `mutex`-ы уходят в kernel space, хочется что-нибудь подешевле.

</p>
</details>

<br />

Вариант double checked locking pattern:

```c++
class Singleton
{
    static Singleton* object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        if (!object)  // check 1
        {
            std::lock_guard guard(mtx);

            if (!object)  // check 2
                object = new Singleton;
        }
        return object;
    }
};
```

**Вопросы**: (чтобы тщательно разобрать код)
* зачем нужна вторая проверка, будет ли код корректен без неё?
* корректен ли код?

<br />

Со временем в этом шаблоне нашли хитрые race condition, что отчасти и подтолкнуло сообщество к формализации memory model в языках Java и С++.
* до Java 2004 DCLP не работал [(подробности)](http://www.cs.umd.edu/~pugh/java/memoryModel/DoubleCheckedLocking.html)
* до С++11 не было портируемого способа реализовать DCLP (стандарт не давал необходимых для этого инструментов и гарантий) 

Проблема в этой строчке:
    
```c++
object = new Singleton;
```

Программист мог бы подумать, что этот код выполняется в такой последовательности:
* выделить память под `Singleton`
* позвать конструктор `Singleton`
* записать указатель в `object`

Здесь в игру вступает instructions reordering.

Например, если компилятор может доказать, что конструктор `Singleton` не бросает исключений, он вправе сгенерировать такой код:

```c++
class Singleton
{
    static Singleton* object = nullptr;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        if (!object)  // check 1
        {
            std::lock_guard guard(mtx);

            if (!object)  // check 2
            {
                object = operator new(sizeof(Singleton));
                new (object) Singleton;
            }
        }
        return object;
    }
};
```

**Вопрос:** в чём здесь проблема? как её можно поймать?

Как это пытались починить различными способами до стандартизации модели памяти, и почему они не работают - [в статье от Мейерса и Александреску](https://www.aristeia.com/Papers/DDJ_Jul_Aug_2004_revised.pdf)

<br />

Корректная реализация double checked locking pattern будет выглядеть так:

```c++
class Singleton
{
    static std::atomic<Singleton*> object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        auto* tmp = object.load();

        if (!tmp)  // check 1
        {
            std::lock_guard<std::mutex> lock(mtx);

            tmp = object.load();
            if (!tmp)  // check 2
            {
                tmp = new Singleton;
                object.store(tmp);
            }
        }
        return tmp;
    }
};
```

В этом примере методы `load`/`store` у атомарных типов гарантируют корректный порядок операций между потоками.

Как именно получаются гарантии и как сделать реализацию быстрее - рассмотрим позже, когда поговорим про memory model.

<br />

Если мы хотим ленивую инициализацию синглтона, то 11-ый стандарт даёт два варианта, которые скрывают проблемы DCLP от программиста:

**Вариант 1**: специальная конструкция для ленивых многопоточных синглтонов - `static`-переменные внутри функции:

```c++
class Singleton
{
public:
    static Singleton& instance()
    {
        static Singleton object;
        return object;
    }
};
```

`static`-переменные внутри функций/методов (не путать с глобальными `static`-переменными!):
* инициализируются лениво при первом обращении
* компилятор с поддержкой С++11 обязан сгенерировать потокобезопасную инициализацию переменной. Компилятор не знает, будет этот код использован в многопоточном варианте или в однопоточном, поэтому он всегда генерирует безопасный многопоточный вариант
* компилятор не обязан использовать DCLP, он может выбрать любой способ, который на его взгляд, работает лучше для данного случая, поэтому:

```c++
#include <string>

float get_circle_area(const float radius)
{
    // скорее всего, здесь не будет никаких синхронизаций,
    // но и static в этом месте - лишнее, лучше удалить
    static const float pi = 3.14f;
    return pi * radius * radius;
}

std::string add_12345(const std::string& x)
{
    // скорее всего, здесь будет синхронизация
    static const std::string s = "12345";
    return x + s;
}
```

Закинуть на godbolt.org этот пример, показать.

**Вопрос на понимание**: в чём разница этих трёх решений?

```c++
std::string add_12345_1(const std::string& x)
{
    static const std::string s = "12345";
    return x + s;
}
```

и

```c++
static const std::string s = "12345";

std::string add_12345_2(const std::string& x)
{
    return x + s;
}
```

и

```c++
std::string add_12345_3(const std::string& x)
{
    const std::string s = "12345";
    return x + s;
}
```

<br />

**Вариант 2**: `std::call_once` (since C++11)

https://en.cppreference.com/w/cpp/thread/call_once



```c++
template< class Callable, class... Args >
void call_once( std::once_flag& flag, Callable&& f, Args&&... args );
```

> Executes the Callable object `f` exactly once, even if called concurrently, from several threads.

`std::call_once` позволяет гарантировать, что какая-то работа будет выполнена только один раз в многопоточной среде.

Более подробно про использование:

```c++
// какая-то работа, единоразовое выполнение которой нужно гарантировать
void some_job(int param, int param2) {  /*...*/ }

// где-то завели флажок, была ли выполнена работа
// (специальный тип std::once_flag)
std::once_flag my_flag;

// вызываем std::call_once в любом потоке:
std::vector<std::thread> threads;
for (int i = 0; i != 5; ++i)
    threads.emplace_back([&once_flag, i](){
        /* .. */
        
        std::call_once(once_flag, some_job, i, i + 1);
        
        /* ... */        
    });

for (auto& t : threads)
    t.join();
```

В этом примере гарантировано, что `some_job` *отработает* только единожды (с каким `param` - неизвестно, с которым первым получится). Когда `some_job` *отработает*, флаг `my_flag` будет взведён, и больше `some_job` не будет вызываться.

<br />

**Резюме** по блоку double checked locking pattern:
* instructions reordering может принести неожиданные сюрпризы в многопоточных программах
* гарантии на последовательность операций дают `std::atomic`-операции со стандарта С++11
* если нужно лениво инициализировать глобальную переменную / константу, предпочтительнее использовать `static`-переменную внутри функции / метода, хороший компилятор выберет наиболее предпочтительный способ синхронизации для вашего случая.
* если нужно гарантировать разовое выполнение работы в многопоточной среде - используйте связку `std::call_once + std::once_flag`

Сравнение производительности `static` / `call_once` и других подходов:
    
http://www.modernescpp.com/index.php/thread-safe-initialization-of-a-singleton
    
Спойлер: `static` всех победил

<br />

**Практические упражнения**:
    
* Найти в 2 потока сумму элементов в массиве целых через одну атомарную переменную, через mutex и независимо по памяти между потоками. Сравнить производительность.
* Найти в 2 потока кол-во вхождений элемента в массив через одну атомарную переменную
* Переписать класс, чтобы подсчитывалось кол-во живых объектов этого класса и общее кол-во созданных объектов

```c++
struct Person {
    int age;
    std::string name;
};
```

* Найти в 2 потока максимальный элемент в массиве целых через одну атомарную переменную